## N = 15

In [1]:
import numpy as np
import torch

print(torch.cuda.is_available())

# load date
X = np.load("Datasets/kryptonite-15-X.npy")
X.shape

True


(30000, 15)

In [2]:
y = np.load("Datasets/kryptonite-15-y.npy")
y.shape

#X = np.load("Datasets/kryptonite-18-X.npy"), y = np.load("Datasets/kryptonite-18-y.npy") shape of X = (36000, 18), y = (36000), 36000 samples, 18 features, 1 target
#I want to calculate the heat map of the correlation between the features and the target

(30000,)

In [ ]:
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import models

# Convert numpy arrays to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float)
y_tensor = torch.tensor(y, dtype=torch.float)

# create a TensorDataset
dataset = TensorDataset(X_tensor, y_tensor)

# define split sizes (60% train, 20% validation, 20% test)
train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Split the dataset into train, validation, and test
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoaders for each subset
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# check data loader output
for X_batch, y_batch in tqdm(train_loader):
    # Reshape X_batch to have a single channel  # Add a channel dimension
    print(X_batch.shape, y_batch.shape)  # X_batch is of shape [batch_size, 1, 15]
    break

class SimpleCNNModel(nn.Module):
    def __init__(self):
        super(SimpleCNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(in_features=128 * 1, out_features=64)
        self.dropout = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(in_features=64, out_features=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool3(x)
        x = x.view(x.size(0), -1)  # Flatten the feature maps
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# initialize the model, loss function, and optimizer
# Initialize the ResNet18 model
# model = CustomResNet18()
# criterion = nn.BCELoss()  # 使用二元交叉熵损失
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    # 使用tqdm显示进度条
    train_pbar = tqdm(train_loader, desc='Training')
    for batch_idx, (inputs, targets) in enumerate(train_pbar):
        inputs, targets = inputs.to(device), targets.to(device)
        inputs = inputs.view(inputs.size(0), 1, -1)
        # print(inputs.shape)
        
        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # 计算准确率
        predictions = (outputs >= 0.5).float()
        correct += (predictions == targets).sum().item()
        total += targets.size(0)
        
        # 更新进度条信息
        train_pbar.set_postfix({
            'loss': f'{total_loss/(batch_idx+1):.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })
    
    return total_loss / len(train_loader), correct / total

def validate(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        val_pbar = tqdm(val_loader, desc='Validation')
        for batch_idx, (inputs, targets) in enumerate(val_pbar):
            inputs, targets = inputs.to(device), targets.to(device)
            inputs = inputs.view(inputs.size(0), 1, -1)
            
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, targets)
            
            total_loss += loss.item()
            predictions = (outputs >= 0.5).float()
            correct += (predictions == targets).sum().item()
            total += targets.size(0)
            
            val_pbar.set_postfix({
                'loss': f'{total_loss/(batch_idx+1):.4f}',
                'acc': f'{100.*correct/total:.2f}%'
            })
    
    return total_loss / len(val_loader), correct / total

torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCNNModel().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
epochs = 100
best_acc = 0
for epoch in range(epochs):
    print(f'\nEpoch {epoch+1}/{epochs}')
    
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    
    print(f'Train Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}%')
    print(f'Val Loss: {val_loss:.4f} | Val Acc: {val_acc*100:.2f}%')
    
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), 'best_model_24.pth')
        print('Saved best model!')


# set the number of epochs
# epochs = 100
# best_acc = 0

# for epoch in range(epochs):
#     """ Training """
#     model.train()

#     # forward pass
#     correct = 0
#     total = 0
#     train_loss = 0
#     for X_batch, y_batch in train_loader:
#         # ResNet18 expects 3-channel images, so we need to expand our data
#         print(X_batch.shape)
#         output = model(X_batch)
#         print("!!!!!!!!")
#         print(output.shape)
#         print(output)
#         y_preds = output.squeeze()

#         correct += torch.eq(y_preds, y_batch).sum().item()
#         total += len(y_batch)

#         loss = loss_fn(output, y_batch)
#         train_loss += loss.item()
#         # zero the optimizer
#         optimizer.zero_grad()
#         # backpropagation
#         loss.backward()
#         # Gradient Descent
#         optimizer.step()

#     """ Validation """
#     model.eval()
#     correct = 0
#     total = 0
#     val_loss = 0
#     with torch.inference_mode():
#         for X_batch, y_batch in val_loader:
#             # ResNet18 expects 3-channel images, so we need to expand our data
#             output = model(X_batch).squeeze()
#             y_preds = torch.round(torch.sigmoid(output))

#             correct += torch.eq(y_preds, y_batch).sum().item()
#             total += len(y_batch)

#             loss = loss_fn(output, y_batch)
#             val_loss += loss.item()
        
#         val_acc = (correct / total) * 100
#         if val_acc > best_acc:
#             best_acc = val_acc
#             torch.save(model.state_dict(), "n-15best.pth")

#     if epoch % 10 == 0:
#         print(f"Epoch: {epoch} | Train Loss: {train_loss:.5f} | Acc: {train_acc:.2f}% | Learning Rate: {scheduler.get_last_lr()[0]:.7f} | Val loss: {val_loss:.5f} | Val Acc: {val_acc:.2f}%")
#     train_acc = (correct / total) * 100

  0%|          | 0/282 [00:00<?, ?it/s]


torch.Size([64, 15]) torch.Size([64])

Epoch 1/100


Validation: 100%|██████████| 94/94 [00:00<00:00, 470.81it/s, loss=0.6931, acc=50.02%]


Train Loss: 0.6934 | Train Acc: 50.36%
Val Loss: 0.6931 | Val Acc: 50.02%
Saved best model!

Epoch 2/100


Validation: 100%|██████████| 94/94 [00:00<00:00, 487.98it/s, loss=0.6932, acc=49.98%]


...

Validation: 100%|██████████| 94/94 [00:00<00:00, 539.77it/s, loss=0.7130, acc=50.83%]


Train Loss: 0.6655 | Train Acc: 59.62%
Val Loss: 0.7130 | Val Acc: 50.83%

Epoch 41/100


Validation: 100%|██████████| 94/94 [00:00<00:00, 543.39it/s, loss=0.7174, acc=50.65%]


Train Loss: 0.6634 | Train Acc: 59.94%
Val Loss: 0.7174 | Val Acc: 50.65%

Epoch 42/100


Training:  51%|█████     | 143/282 [00:00<00:00, 287.10it/s, loss=0.6621, acc=60.14%]